In [58]:
import ollama
import json

In [53]:
SYSTEM_PROMPT = """
You are an Intelij based IDE crash/error log parser. Given any IDE fatal error log, identify its log type, then extract a crash fingerprint as a JSON object.

## Step 1 — Identify the log type

| Type | How to recognize |
|---|---|
| `hs_err` | Starts with `# A fatal error has been detected by the Java Runtime Environment` |
| `oom` | Contains `There is insufficient memory for the Java Runtime Environment` |
| `exception` | Starts with `Exception in <ThreadName>:` followed by a Java exception class and stack trace |
| `unknown` | None of the above match, or the input is empty |

## Step 2 — Extract the following fields

### `exception_type`
The error signal, exception, or condition name. Derive from:

| Log type | Source | Example result |
|---|---|---|
| `hs_err` | `#  <NAME> (0x...)` line | `SIGSEGV`, `EXCEPTION_ACCESS_VIOLATION` |
| `oom` | Fixed value | `OutOfMemoryError` |
| `exception` | The Java exception class on the second line | `java.lang.IllegalArgumentException` |
| `unknown` | — | `null` |

### `frame_type`
Only applicable to `hs_err` logs. Single character from the "Problematic frame" line:
`C` (native), `J` (JIT Java), `V` (JVM internal), `j` (interpreted Java).
Set to `null` for all other log types.

### `frame_fingerprint`
A stable, de-noised identifier. Strip all hex addresses, numeric offsets (`+0x...`),
byte counts, argument signatures, and return types.

| Log type | Source | Derive from | Example result |
|---|---|---|---|
| `hs_err` (C frame) | Problematic frame | library name + function name (if present) | `libsystem_kernel.dylib::__kill` or `chrome_elf.dll` |
| `hs_err` (J frame) | Problematic frame | fully qualified class + method name | `com.example.Foo.parseLogLine` |
| `hs_err` (V frame) | Problematic frame | library name + C++ class::method (no args) | `libjvm.dylib::G1CMTask::process_grey_task_entry` |
| `oom` | `Out of Memory Error (<file>:<line>)` | source file + line number | `arena.cpp:168` |
| `exception` | First Java frame in the stack trace (skip native frames) | fully qualified class + method name | `sun.lwawt.macosx.CInputMethod.insertText` |
| `unknown` | — | — | `null` |

## Output
Return only valid JSON, no explanation:
{
  "log_type": "...",
  "exception_type": "...",
  "frame_type": "...",
  "frame_fingerprint": "..."
}
"""

In [54]:
# List available models
models = ollama.list()
print(f"Available models:")
for model in models['models']:
    print(model['model'])

Available models:
qwen3:4b-instruct
qwen3:4b


In [55]:
# Using different models
DEFAULT_MODEL = 'qwen3:4b'

log_to_analyze = ''.join(open('java_error_in_devecostudio64_36492.log', encoding='gbk').readlines()[:100])
question = f"\n\nThe log to analyze is as follows, output the crash fingerprint in JSON format:\n\n" + log_to_analyze

try:
    response = ollama.chat(
        model=DEFAULT_MODEL,
        messages=[
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': question}
        ],
        stream=True,
        options={'temperature': 0.1, 'format': 'json'}
    )
    answer = ''
    for chunk in response:
        #print(chunk['message']['thinking'], end='', flush=True)
        token = chunk['message']['content']
        answer += token
        print(token, end='', flush=True)
    print()  # New line after response is complete
except Exception as e:
    print(f"Error: {e}")

{
  "log_type": "oom",
  "exception_type": "OutOfMemoryError",
  "frame_type": null,
  "frame_fingerprint": "arena.cpp:79"
}


In [59]:
json.loads(answer)

{'log_type': 'oom',
 'exception_type': 'OutOfMemoryError',
 'frame_type': None,
 'frame_fingerprint': 'arena.cpp:79'}